In [1]:
import os

In [2]:
project_name = 'benchmark_autodl'

Creates a project, with project name

In [3]:
os.popen('mkdir {}'.format(project_name))

Copies the model_script.py to the folder, please give the right path to model_script.py, if you are using from git, you will find it in the folder 'Exemple_AutoDl_project_folder' Attention !!!

In [4]:
python_script = os.path.join("/", "C:", os.sep, 'Users', "azizeac", "nni",
                                 "examples", "trials",
                                 "benchmark_deep", "model_script.py")
project_directory = os.path.join(os.popen("echo %CD%").read()[:-1], project_name)
os.popen('copy {} {}'.format(python_script,project_directory))

Creates the search_space.json from the dictionaries

In [5]:
import json

search_space = {
  "hidden_size": {
    "_type": "choice",
    "_value": [ 1000, 750, 500, 250 ]
  },
  "lr": {
    "_type": "choice",
    "_value": [ 0.0001, 0.001, 0.01, 0.1 ]
  },
  "momentum": {
    "_type": "uniform",
    "_value": [ 0.8, 1 ]
  },
  "ps1": {
    "_type": "uniform",
    "_value": [ 0, 0.2 ]
  },
  "ps2": {
    "_type": "uniform",
    "_value": [ 0, 0.1 ]
  },
  "emb_drop": {
    "_type": "uniform",
    "_value": [ 0, 0.1 ]
  }
}


with open('search_space.json', 'w') as outfile:
    json.dump(search_space, outfile)
    os.popen('copy search_space.json {}'.format(project_directory))

In [6]:
datasets = ['albert','christine','jasmine','banking','bnp_cardif_claims','home-credit-default-risk','DADGP']
targets = ['class','class','class','subscribed','target','TARGET','target']

Creates the config.yml file

In [7]:
import yaml
config_script = os.path.join("/", "C:", os.sep, 'Users', "azizeac", "nni",
                                 "examples", "trials",
                                 "benchmark_deep", "config_windows.yml")
with open(config_script) as f:
     list_yaml = yaml.load(f, Loader=yaml.FullLoader)

data_dir = data_dir_path = os.path.join("/", "N:", os.sep, 'Projets01', "MLBox", "AutoML_datasets_for_MLBox",
                                 "Financial_public_datasets", "CA_public_datasets",
                                 "home-credit-default-risk","application_train.csv")

target = 'TARGET'

max_duration = '30mn'
maxTrialNumber = '100'
tuner = 'TPE'

list_yaml['maxExecDuration'] = max_duration
list_yaml['tuner']['builtinTunerName'] = tuner
list_yaml['trial']['command'] = "python model_script.py {} {}".format(data_dir, target)

In [8]:
with open("config.yml", "w") as f:
    yaml.dump(list_yaml, f)
    os.popen('copy config.yml {}'.format(project_directory))

Starts the experiment !

In [ ]:
stream_start = os.popen('nnictl create --config {} --port 8081'.format(os.path.join(project_directory,"config.yml"))).read()

In [16]:
stream_start

"INFO:  expand searchSpacePath: search_space.json to C:\\Users\\azizeac\\notebook\\project_2\\search_space.json \nINFO:  expand codeDir: . to C:\\Users\\azizeac\\notebook\\project_2\\. \nERROR: Port 8080 is used by another process, please reset the port!\nYou could use 'nnictl create --help' to get help information\n"

In [13]:
stream_stop = os.popen('nnictl stop').read()

In [14]:
stream_stop

'ERROR: There is no experiment running...\n'

Retrieve informations about the experiments !

In [ ]:
nc.set_endpoint('http://127.0.0.1:8080')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

jobs = nc.list_trial_jobs()
job_ids = [x['id'] for x in jobs]
final_metrics = [x['finalMetricData'][0]['data'][1:-1] if 'finalMetricData' in x.keys() else 0 for x in jobs ]

data = {'job id': job_ids, 'final metrics': final_metrics}
sns.set(rc={'figure.figsize':(15, 6)})

plt.title('Trial job final results')
ax = sns.barplot(x='job id', y='final metrics', data=data) 

for i,p in enumerate(ax.patches):
    ax.annotate('{:.4f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=11, color='black', rotation=0, xytext=(0, 5),
                 textcoords='offset points') 

In [ ]:
nc.stop_nni()